# Chapter 18

## 18.1
实现一个Bug类，对沿着水平线爬行的虫子建模。move方法向当前方向移动，turn方法让虫子转身，show方法打印出当前的位置。让这些方法可以被串接调用。例如：
```
bugsy.move(4).show().move(6).show().turn().move(5).show()
```
上述代码应显示 4 10 5。

In [1]:
class Bug(var pos: Int = 0) {
    var forward: Int = 1
    
    def move(n: Int):this.type = {
        pos += forward * n
        this
    }
    
    def show():this.type = {
        println("current position: " + pos)
        this
    }
    
    def turn():this.type = {
        forward = -forward
        this
    }
}

val bugsy = new Bug
bugsy.move(4).show().move(6).show().turn().move(5).show()

current position: 4
current position: 10
current position: 5


defined class Bug
bugsy: Bug = $sess.cmd0Wrapper$Helper$Bug@2e8d9f13
res0_2: Bug = $sess.cmd0Wrapper$Helper$Bug@2e8d9f13

## 18.2
为前一个练习中的Bug类提供一个流利接口，达到能编写如下代码的效果：
```
bugsy move 4 and show and then move 6 and show turn around move 5 and show
```

In [23]:
abstract class BugAction
case object show extends BugAction
case object then extends BugAction
case object around extends BugAction

class Bug(var pos: Int = 0) {
    var forward: Int = 1
    
    def move(num: Int): this.type = { pos += forward * num; this }

    def turn(obj: around.type): this.type = { forward = -forward; this}
    
    def and(act: BugAction): this.type = {
        act match {
            case s: show.type => { print(pos + " "); this }
            case t: then.type => this
        }
    }
}

val bugsy = new Bug
bugsy move 4 and show and then move 6 and show turn around move 5 and show

4 10 5 

defined class BugAction
defined object show
defined object then
defined object around
defined class Bug
bugsy: wrapper.wrapper.Bug = $sess.cmd22Wrapper$Helper$Bug@53233476
res22_6: wrapper.wrapper.Bug = $sess.cmd22Wrapper$Helper$Bug@53233476

## 18.3
完成18.1节中的流利接口，以便我们可以做出如下调用：
book set Title to “Scala for the Impatient” set Author to “Cay Horstmann”

In [24]:
class Document {
    var title: String = ""
    var author: String = ""
    def setTitle(title: String):this.type = {this.title = title; this}
    def setAuthor(author: String):this.type = {this.author= author; this}
}

class BookAction
case object Title extends BookAction
case object Author extends BookAction

class Book extends Document{
    private var nextAction: BookAction = _
    
    def set(action: BookAction): this.type = {
        action match {
            case Title => { nextAction = Title; this }
            case Author => { nextAction = Author; this }
        }
    }
    
    def to(arg: String): this.type = {
      nextAction match {
        case Title => { title = arg; this }
        case Author => { author = arg; this }
      }
    }
}

val book = new Book
book set Title to "Scala for the Impatient" set Author to "Cay Horstmann"
println(book.title)
println(book.author)

Scala for the Impatient
Cay Horstmann


defined class Document
defined class BookAction
defined object Title
defined object Author
defined class Book
book: Book = $sess.cmd23Wrapper$Helper$Book@41bae2b
res23_6: Book = $sess.cmd23Wrapper$Helper$Book@41bae2b

## 18.4
实现18.2节中被嵌套在Network类中的Member类的equals方法。两个成员要想相等，必须属于同一个网络。

In [28]:
import scala.collection.mutable.ArrayBuffer

class Network {
    class Member(val name: String) {
        val contacts = new ArrayBuffer[Member]
        
        override def equals(other: Any): Boolean = {
            other match {
                case that: Member => {
                    that.isInstanceOf[Member] && name == that.name
                }
                case _ => false
            }
        }
    }
    
    private val members = new ArrayBuffer[Member]
    
    def join(name: String) = {
        val m = new Member(name)
        members += m
        m
    }
}

val network1 = new Network().join("1")
val network2 = new Network().join("1")
network1.equals(network2)

import scala.collection.mutable.ArrayBuffer


defined class Network
network1: wrapper.wrapper.Network#Member = $sess.cmd27Wrapper$Helper$Network$Member@2c9352fc
network2: wrapper.wrapper.Network#Member = $sess.cmd27Wrapper$Helper$Network$Member@c2ef256
res27_4: Boolean = false

## 18.5
考虑如下类型别名
```
type NetworkMember = n.Member forSome { val n : Network }
```
和函数
```
def process(m1: NetworkMember, m2: NetworkMember) = (m1, m2)
```
这与18.8节中的process函数有什么不同？

A: process接受相同或不同网络的成员, 而18.8节中的函数(process)则拒绝那些来自不同网络的成员。

## 18.6
Scala类库中的Either类型可以被用于要么返回结果，要么返回某种失败信息的算法。编写一个带有两个参数的函数：一个已排序整数数组和一个整数值。要么返回该整数值在数组中的下标，要么返回最接近该值的元素的下标。使用一个中置类型作为返回类型。

In [30]:
def getIndex(arr: Seq[Int], v: Int): Int Either Int = {
    if (arr.contains(v)) {
        Left(arr.indexOf(v))
    } else {
        Right(arr.indexOf(arr.reduce((a, b) => if (math.abs(a - v) > math.abs(b - v)) b else a)))
    }
}

getIndex(Seq(2,3,4,5,-2,-3),6)
getIndex(Seq(2,3,4,5,-2,-3),3)

defined function getIndex
res29_1: Either[Int, Int] = Right(3)
res29_2: Either[Int, Int] = Left(1)

## 18.7
实现一个方法，接受任何具备如下方法的类的对象和一个处理该对象的函数。调用该函数，并在完成或有任何异常发生时调用close方法。
```
def close(): Unit
```

In [31]:
def tryWithClose[T <: {def close(): Unit}](obj: T, func: T => Unit) {
    try {
        func(obj)
    } finally {
        obj.close()
    }
}

defined function tryWithClose

## 18.8
编写一个函数printValues，带有三个参数f、from和to，打印出所有给定区间范围内的输入值经过f计算后的结果。这里的f应该是任何带有接受Int产出Int的apply方法的对象。例如：
```
printValues((x: Int) => x*x, 3, 6) //将打印 9 16 25 36
printValues(Array(1, 1, 2, 3, 5, 8, 13, 21, 34, 55), 3, 6) //将打印 3 5 8 13
```

In [34]:
def printValues(f: {def apply(param:Int):Int}, from: Int, to: Int) {
    for(i <- from to to) {
        print(f.apply(i) + " ")
    }
    println()
}

printValues((x: Int) => x*x, 3, 6)

printValues(Array(1, 1, 2, 3, 5, 8, 13, 21, 34, 55), 3, 6)

9 16 25 36 
3 5 8 13 


defined function printValues

## 18.9
考虑如下对物理度量建模的类
```scala
abstract class Dim[T](val value: Double, val name: String) {
    protected def create(v: Double): T
    def +(other: Dim[T]) = create(value + other.value)
    override def toString() = value + " " + name
}
```
以下是具体子类:
```scala
class Seconds(v: Double) extends Dim[Seconds] (v, "s") {
    override def create(v: Double) = new Seconds(v)
}
```
但不清楚状况的人可能会定义
```scala
class Meters(v: Double) extends Dim[Meters](v, "m") {
    override def create(v: Double) = new Seconds(v)
}
```
允许米（Meters）和秒（Seconds）相加。使用自身类型来防止发生这样的情况。

In [36]:
/*
public abstract class Dim<T>
{
    private final double value;
    private final String name;

    public double value() {
        return this.value;
    }

    public String name() {
        return this.name;
    }

    public abstract T create(final double v);

    public T $plus(final Dim<T> other) {
        return this.create(this.value() + other.value());
    }

    @Override
    public String toString() {
        return this.value() + " " + this.name();
    }

    public Dim(final double value, final String name) {
        this.value = value;
        this.name = name;
    }
}
*/
abstract class Dim[T](val value: Double, val name: String){
  this: T =>
  protected def create(v: Double): T
  def + (other: Dim[T]) = create(value + other.value)
  override def toString() = value + " " + name
}

/*
public class Seconds extends Dim<Seconds>
{
    @Override
    public Seconds create(final double v) {
        return new Seconds(v);
    }

    public Seconds(final double v) {
        super(v, "s");
    }
}
*/
class Seconds(v: Double) extends Dim[Seconds](v, "s"){
  override def create(v: Double) = new Seconds(v)
}
class Meters(v: Double) extends Dim[Seconds](v, "m"){
  override def create(v: Double) = new Seconds(v)
}

val s = new Seconds(10)
val m = new Meters(5)
s + m

cmd36.sc:10: illegal inheritance;
 self-type Helper.this.Meters does not conform to Helper.this.Dim[Helper.this.Seconds]'s selftype Helper.this.Dim[Helper.this.Seconds] with Helper.this.Seconds
class Meters(v: Double) extends Dim[Seconds](v, "m"){
                                ^

: 

## 18.10
自身类型通常可以被扩展自身的特质替代，但某些情况下使用自身类型会改变初始化和重写的顺序。构造出这样的一个示例。

In [45]:
trait A {
    def sing() = "from a"
}

trait C {
    this: A =>
    val w = sing + "from c"
}

class B {
    this: C =>
    val k = w
}

val b = new B with C with A
// val b = new B with A with C
println(b.k)

null


defined trait A
defined trait C
defined class B
b: wrapper.wrapper.B with wrapper.wrapper.A with wrapper.wrapper.C = $sess.cmd44Wrapper$Helper$$anon$1@5f3f1dc